# SYMFLUENCE Tutorial 02c — Basin-Scale Workflow (Bow River at Banff, Elevation-Based Distributed)

## Introduction

This tutorial demonstrates the most spatially-detailed modeling approach: elevation-based HRU discretization. Building on Tutorials 02a (lumped) and 02b (semi-distributed), we now subdivide each GRU into elevation bands that capture altitudinal controls on mountain hydrology.

Elevation-based discretization is critical in mountain watersheds where temperature and precipitation vary systematically with altitude. By stratifying each sub-basin into elevation bands, we better represent snowpack dynamics, seasonal timing differences, and orographic effects.

The key configuration parameter is `ELEVATION_BAND_SIZE`, which controls the vertical resolution (e.g., 100m bands). Smaller bands increase spatial detail but add computational cost. This approach maintains the validated GRU structure from Tutorial 02b while adding vertical stratification.

For the **Bow River at Banff** (elevation range: 1,384–3,400 m), elevation bands capture the transition from low-elevation rain-dominated zones to high-elevation snow-dominated headwaters, improving simulation of spring freshet timing and runoff generation.


# Step 1 — Configuration and data reuse

We configure elevation-based discretization and reuse data from Tutorial 02b where possible.

In [ ]:
# Step 1 — Elevation-based configuration with data reuse

from pathlib import Path
import yaml
import shutil
import sys
sys.path.append(str(Path("../..").resolve()))
from SYMFLUENCE import SYMFLUENCE

# Define directories
SYMFLUENCE_CODE_DIR = Path("../..").resolve()
SYMFLUENCE_DATA_DIR = Path("/Users/darrieythorsson/compHydro/test/SYMFLUENCE_data").resolve()

# Load template
config_template = SYMFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'
with open(config_template, 'r') as f:
    config = yaml.safe_load(f)

# === Modify for elevation-based distributed ===
config['SYMFLUENCE_CODE_DIR'] = str(SYMFLUENCE_CODE_DIR)
config['DOMAIN_NAME'] = 'Bow_at_Banff_elevation'
config['EXPERIMENT_ID'] = 'run_1'
config['POUR_POINT_COORDS'] = '51.1722/-115.5717'

# Elevation-based discretization
config['DOMAIN_DEFINITION_METHOD'] = 'delineate'
config['STREAM_THRESHOLD'] = 5000  # Same as 02b
config['DOMAIN_DISCRETIZATION'] = 'elevation'  # Key change
config['ELEVATION_BAND_SIZE'] = 400  # 400m elevation bands


config['HYDROLOGICAL_MODEL'] = 'SUMMA'
config['ROUTING_MODEL'] = 'mizuRoute'
config['MIZU_FROM_MODEL'] = 'SUMMA'


config['SETTINGS_MIZU_ROUTING_VAR'] = 'averageRoutedRunoff'
config['SETTINGS_MIZU_ROUTING_UNITS'] = 'm/s'
config['SETTINGS_MIZU_ROUTING_DT'] = '3600'

# Temporal extent
config['EXPERIMENT_TIME_START'] = '2004-01-01 01:00'
config['EXPERIMENT_TIME_END'] = '2007-12-31 23:00'
config['CALIBRATION_PERIOD'] = '2005-10-01, 2006-09-30'
config['EVALUATION_PERIOD'] = '2006-10-01, 2007-12-30'
config['SPINUP_PERIOD'] = '2004-01-01, 2005-09-30'

config['STATION_ID'] = '05BB001'
config['DOWNLOAD_WSC_DATA'] = True

config['PARAMS_TO_CALIBRATE'] = 'minStomatalResistance,cond2photo_slope,vcmax25_canopyTop,jmax25_scale,summerLAI,rootingDepth,soilStressParam,z0Canopy,windReductionParam'
config['OPTIMISATION_TARGET'] = 'streamflow'
config['ITERATIVE_OPTIMIZATION_ALGORITHM'] = 'DDS'
config['OPTIMIZATION_METRIC'] = 'KGE'
config['CALIBRATION_TIMESTEP'] = 'hourly'  


# Save configuration
config_path = SYMFLUENCE_CODE_DIR / '0_config_files' / 'config_elevation_distributed.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved: {config_path}")

# === Data reuse from Tutorial 02b ===
semi_dist_domain = 'Bow_at_Banff_semi_distributed'
semi_dist_data_dir = SYMFLUENCE_DATA_DIR / f'domain_{semi_dist_domain}'

def copy_with_name_adaptation(src, dst, old_name, new_name):
    """Copy directory and adapt filenames"""
    if not src.exists():
        return False
    dst.parent.mkdir(parents=True, exist_ok=True)
    if src.is_file():
        shutil.copy2(src, dst)
        return True
    shutil.copytree(src, dst, dirs_exist_ok=True)
    for file in dst.rglob('*'):
        if file.is_file() and old_name in file.name:
            new_file = file.parent / file.name.replace(old_name, new_name)
            file.rename(new_file)
    return True

# Initialize SYMFLUENCE
symfluence = SYMFLUENCE(config_path)
project_dir = symfluence.managers['project'].setup_project()

if semi_dist_data_dir.exists():
    print(f"\n📋 Reusing data from Tutorial 02b: {semi_dist_data_dir}")
    
    reusable_data = {
        'Elevation': semi_dist_data_dir / 'attributes' / 'elevation',
        'Land Cover': semi_dist_data_dir / 'attributes' / 'landclass',
        'Soils': semi_dist_data_dir / 'attributes' / 'soilclass',
        'Forcing': semi_dist_data_dir / 'forcing' / 'raw_data',
        'Stream Network': semi_dist_data_dir / 'shapefiles' / 'river_network',
        'GRUs': semi_dist_data_dir / 'shapefiles' / 'river_basins',
        'Streamflow': semi_dist_data_dir / 'observations' / 'streamflow'
    }
    
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            rel_path = src_path.relative_to(semi_dist_data_dir)
            dst_path = project_dir / rel_path
            success = copy_with_name_adaptation(src_path, dst_path, semi_dist_domain, config['DOMAIN_NAME'])
            if success:
                print(f"   ✅ {data_type}: Copied")
        else:
            print(f"   📋 {data_type}: Not found")
else:
    print(f"\n⚠️  No data from Tutorial 02b found. Will acquire fresh data.")

# Create pour point
pour_point_path = symfluence.managers['project'].create_pour_point()
print(f"\n✅ Project structure created at: {project_dir}")

## Step 2 — Elevation-based discretization

Subdivide each GRU from Tutorial 02b into elevation bands for vertical stratification.

### Step 2a — Attribute check

Verify DEM and GRU availability from data reuse.

In [ ]:
# Step 2a — DEM and GRU availability check
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
gru_path = project_dir / 'shapefiles' / 'river_basins'

if not dem_path.exists() or not gru_path.exists():
    print("   Required data not found, acquiring...")
    # If using MAF supported HPC, uncomment the lines below
    # symfluence.managers['data'].acquire_attributes()
    # symfluence.managers['domain'].define_domain()
    print("✅ Geospatial data acquired")
else:
    print("✅ DEM and GRU data available from previous workflow")

### Step 2b — Elevation band creation

Create HRUs by intersecting GRUs with elevation bands.

In [ ]:
# Step 2b — Elevation-based HRU discretization
hru_path = symfluence.managers['domain'].discretize_domain()
print("✅ Elevation-based HRU discretization complete")

### Step 2c — Elevation structure visualization

Visualize the elevation-stratified HRU structure.

In [ ]:
# Step 2c — Elevation band visualization

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# Load HRUs with elevation information
hru_file = project_dir / 'shapefiles' / 'catchment' / f"{config['DOMAIN_NAME']}_HRUs_elevation.shp"

if hru_file.exists():
    hru_gdf = gpd.read_file(str(hru_file))
    print(f"Number of elevation-based HRUs: {len(hru_gdf)}")
    
    # Calculate elevation statistics
    if 'elev_mean' in hru_gdf.columns:
        elev_col = 'elev_mean'
    elif 'elevation' in hru_gdf.columns:
        elev_col = 'elevation'
    else:
        elev_col = hru_gdf.select_dtypes(include=[np.number]).columns[0]
    
    print(f"Elevation range: {hru_gdf[elev_col].min():.0f} - {hru_gdf[elev_col].max():.0f} m")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Spatial map colored by elevation
    hru_gdf.plot(column=elev_col, cmap='terrain', ax=axes[0], 
                 legend=True, legend_kwds={'label': 'Elevation (m)'})
    
    pour_point_gdf = gpd.read_file(pour_point_path)
    pour_point_gdf.plot(ax=axes[0], color='red', markersize=150, marker='*', label='Pour Point')
    
    axes[0].set_title(f'Elevation-Based HRU Distribution\n{len(hru_gdf)} HRUs', fontweight='bold')
    axes[0].set_xlabel('Longitude')
    axes[0].set_ylabel('Latitude')
    axes[0].legend()
    
    # Elevation distribution histogram
    axes[1].hist(hru_gdf[elev_col], bins=20, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Elevation (m)')
    axes[1].set_ylabel('Number of HRUs')
    axes[1].set_title('HRU Elevation Distribution', fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  HRU shapefile not found")

## Step 3 — Data preprocessing

Process forcing and observation data for elevation-stratified HRUs.

In [ ]:
# Step 3a — Streamflow observations
# If using MAF supported HPC, uncomment the line below
# symfluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

In [ ]:
# Step 3b — Forcing data
# If using MAF supported HPC, uncomment the line below
# symfluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete")

In [ ]:
# Step 3c — Model-agnostic preprocessing
symfluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

## Step 4 — Model execution

Configure and run SUMMA-mizuRoute with elevation-stratified HRUs.

In [ ]:
# Step 4a — Model configuration
symfluence.managers['model'].preprocess_models()
print("✅ Elevation-based model configuration complete")

In [ ]:
# Step 4b — Model execution
print(f"Running {config['HYDROLOGICAL_MODEL']} with {config['ROUTING_MODEL']} ({len(hru_gdf)} elevation-based HRUs)...")
symfluence.managers['model'].run_models()
print("✅ Elevation-based distributed simulation complete")

## Step 5 — Evaluation

Compare elevation-based results against observations and previous approaches.

In [ ]:
# Step 5 — Elevation-based HRUs evaluation 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# ---------------------------
# 1) Observations (daily, tz-naive)
# ---------------------------
obs_path = project_dir / "observations" / "streamflow" / "preprocessed" / f"{config['DOMAIN_NAME']}_streamflow_processed.csv"
obs_df = pd.read_csv(obs_path, parse_dates=["datetime"]).set_index("datetime").sort_index()

# Make tz-naive if needed
if obs_df.index.tz is not None:
    obs_df.index = obs_df.index.tz_convert(None)

# Put obs on daily mean (change to .sum() if your obs file is daily volumes)
obs_daily = obs_df["discharge_cms"].resample("D").mean().rename("discharge_obs")

# ---------------------------
# 2) Simulation (select segment, daily, tz-naive)
# ---------------------------
routing_dir = project_dir / "simulations" / config["EXPERIMENT_ID"] / "mizuRoute"
sim_files = list(routing_dir.glob("*.nc"))  # allow any routed filename
if not sim_files:
    raise FileNotFoundError(f"No routed streamflow in: {routing_dir}")

ds = xr.open_dataset(sim_files[0], decode_times=False)

# Fix non-CF units on the time coordinate if needed
time_name = next((d for d in ds.dims if 'time' in d.lower()), 'time')
tvar = ds[time_name]
units = (tvar.attrs or {}).get('units', '')
cal = (tvar.attrs or {}).get('calendar', 'standard')

def _parse_ref_date(u):
    # Accepts "s since 1990-1-1 0:0:0" or "seconds since 1990-01-01 00:00:00"
    if 'since' not in u:
        return None, None
    unit_part, ref_part = u.split('since', 1)
    unit_part = unit_part.strip().lower()
    ref_part = ref_part.strip()
    # Normalize short forms
    unit_norm = {'s': 'seconds', 'sec': 'seconds', 'secs': 'seconds'}.get(unit_part, unit_part)
    return unit_norm, ref_part

unit_norm, ref = _parse_ref_date(units)

if unit_norm == 'seconds' and ref is not None:
    # Build a pandas datetime index
    ref_ts = pd.to_datetime(ref)
    # ensure numeric array (int/float)
    tvals = xr.apply_ufunc(lambda x: x.astype('float64'), tvar, dask='allowed')
    new_time = ref_ts + pd.to_timedelta(tvals, unit='s')
    # assign fixed, CF-friendly coordinate
    ds = ds.assign_coords({time_name: ('time', pd.DatetimeIndex(new_time.values))})
    # Optional: clean attrs to a CF-compliant string
    ds[time_name].attrs['units'] = f"seconds since {ref_ts.strftime('%Y-%m-%d %H:%M:%S')}"
    ds[time_name].attrs['calendar'] = cal
else:
    # If units are already CF-compliant or different, you can try decoding now
    try:
        ds = xr.decode_cf(ds)
    except Exception:
        pass  # fall back to naive numeric time if needed

var = ds["IRFroutedRunoff"]

# find time dimension
time_dim = next((d for d in var.dims if "time" in d.lower()), None)
if time_dim is None:
    raise RuntimeError("Could not find a time dimension in IRFroutedRunoff.")

# choose the (first) non-time dimension as the segment dim
seg_dims = [d for d in var.dims if d != time_dim]
if not seg_dims:
    sim_series = var  # 1D over time
else:
    seg_dim = seg_dims[0]

    # 1) count finite points per segment
    good_counts = xr.apply_ufunc(np.isfinite, var).sum(dim=time_dim)

    # 2) keep only segments with at least one finite value
    valid_mask = good_counts > 0
    if not bool(valid_mask.any()):
        raise RuntimeError(
            "All segments are NaN over time in IRFroutedRunoff. "
            "Check that you opened the mizuRoute file (not SUMMA), the variable name, "
            "and that the run produced non-empty routed flows."
        )

    var_valid = var.where(valid_mask, drop=True)

    # 3) select segment with highest mean (over valid data)
    seg_mean = var_valid.mean(time_dim, skipna=True)
    max_idx = seg_mean.argmax(dim=seg_dim)
    seg_id = seg_mean[seg_dim].isel({seg_dim: max_idx}).item()

    # 4) extract the time series for that segment (use original var to keep coordinates)
    sim_series = var.sel({seg_dim: seg_id})
    print(f"[02c] Selected segment dim: {seg_dim}, id: {seg_id}")
    
# Convert to pandas
sim_df = sim_series.to_series().sort_index()
if getattr(sim_df.index, "tz", None) is not None:
    sim_df.index = sim_df.index.tz_convert(None)

# Daily cadence (use .sum() if routed values are volumes per timestep)
sim_daily = sim_df.resample("D").mean().rename("discharge_sim")

# ---------------------------
# 3) Spinup: skip first year from simulation start
# ---------------------------
spinup_end = (sim_daily.index.min() + pd.DateOffset(years=1)).normalize()
print("Spinup ends at:", spinup_end)

obs_daily_trim = obs_daily.loc[obs_daily.index >= spinup_end]
sim_daily_trim = sim_daily.loc[sim_daily.index >= spinup_end]

# ---------------------------
# 4) Align & build evaluation frame
# ---------------------------
eval_df = pd.concat([obs_daily_trim, sim_daily_trim], axis=1) \
             .dropna(subset=["discharge_obs", "discharge_sim"]) \
             .sort_index()

if eval_df.empty:
    raise RuntimeError(
        "No overlap after spinup/alignment. Check segment selection, units (mean vs sum), "
        "timezone, and cadences."
    )

obs_valid = eval_df["discharge_obs"]
sim_valid = eval_df["discharge_sim"]

# ---------------------------
# 5) NaN-safe metrics
# ---------------------------
def _safe_div(a, b):
    return np.nan if (b == 0 or np.isnan(b)) else a / b

def nse(obs, sim):
    num = np.nansum((obs - sim) ** 2)
    den = np.nansum((obs - np.nanmean(obs)) ** 2)
    return float(1 - _safe_div(num, den))

def kge(obs, sim):
    r = np.corrcoef(obs, sim)[0, 1]
    alpha = np.nanstd(sim) / np.nanstd(obs)
    beta = np.nanmean(sim) / np.nanmean(obs)
    return float(1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2))

def pbias(obs, sim):
    num = np.nansum(sim - obs)
    den = np.nansum(obs)
    return float(100 * _safe_div(num, den))

nse_val   = np.round(nse(obs_valid, sim_valid), 3)
kge_val   = np.round(kge(obs_valid, sim_valid), 3)
pbias_val = np.round(pbias(obs_valid, sim_valid), 1)

print("Performance Metrics:")
print(f"  NSE:  {nse_val}")
print(f"  KGE:  {kge_val}")
print(f"  PBIAS:{pbias_val}%")
print(f"  HRUs: {len(hru_gdf)}")

# ---------------------------
# 6) Visualization (same layout)
# ---------------------------
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(eval_df.index, eval_df["discharge_obs"].values, 'b-', label='Observed', linewidth=1.2, alpha=0.7)
axes[0, 0].plot(eval_df.index, eval_df["discharge_sim"].values, 'r-', 
                label=f'Elevation-Based ({len(hru_gdf)} HRUs)', linewidth=1.2, alpha=0.7)
axes[0, 0].set_ylabel('Discharge (m³/s)')
axes[0, 0].set_title('Elevation-Based Streamflow')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].text(0.02, 0.95, f"NSE: {nse_val}\nKGE: {kge_val}\nBias: {pbias_val}%\nHRUs: {len(hru_gdf)}",
                transform=axes[0, 0].transAxes, va='top',
                bbox=dict(facecolor='white', alpha=0.8), fontsize=9)

# Scatter
axes[0, 1].scatter(obs_valid, sim_valid, alpha=0.5, s=10)
mmax = float(max(obs_valid.max(), sim_valid.max()))
axes[0, 1].plot([0, mmax], [0, mmax], 'k--', alpha=0.5)
axes[0, 1].set_xlabel('Observed (m³/s)')
axes[0, 1].set_ylabel('Simulated (m³/s)')
axes[0, 1].set_title('Observed vs Simulated')
axes[0, 1].grid(True, alpha=0.3)

# Monthly climatology
monthly_obs = obs_valid.groupby(obs_valid.index.month).mean()
monthly_sim = sim_valid.groupby(sim_valid.index.month).mean()
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
axes[1, 0].plot(monthly_obs.index, monthly_obs.values, 'b-o', label='Observed', markersize=6)
axes[1, 0].plot(monthly_sim.index, monthly_sim.values, 'r-o', label='Simulated', markersize=6)
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].set_xticklabels(month_names)
axes[1, 0].set_ylabel('Mean Discharge (m³/s)')
axes[1, 0].set_title('Seasonal Flow Regime')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Tutorial progression bar: robust to missing earlier metrics
labels, units, nses = [], [], []
# If you’ve saved/defined earlier metrics, append them here; otherwise we only plot 02c
if "nse_val_02a" in globals():
    labels.append("02a\n(Lumped)"); units.append(1); nses.append(float(nse_val_02a))
if "nse_val_02b" in globals():
    labels.append("02b\n(Semi-Dist)"); units.append(globals().get("gru_count_02b", "N/A")); nses.append(float(nse_val_02b))

labels.append("02c\n(Elevation)")
units.append(len(hru_gdf))
nses.append(float(nse_val))

x_pos = np.arange(len(labels))
bars = axes[1, 1].bar(x_pos, nses, alpha=0.7, edgecolor='k')
axes[1, 1].set_xlabel('Tutorial Progression')
axes[1, 1].set_ylabel('Nash-Sutcliffe Efficiency')
axes[1, 1].set_title('Performance vs Complexity')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels(labels)
axes[1, 1].set_ylim(0, 1)
axes[1, 1].grid(True, alpha=0.3, axis='y')

for bar, units_i, nse_i in zip(bars, units, nses):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., float(nse_i) + 0.02,
                    f'{nse_i:.3f}\n({units_i} units)', ha='center', va='bottom', fontsize=9)

plt.suptitle(f'Elevation-Based Evaluation — {config["DOMAIN_NAME"]} ({len(hru_gdf)} HRUs)',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n✅ Elevation-based evaluation complete")


# Step 5b — Run calibration 



In [ ]:
results_file = symfluence.managers['optimization'].calibrate_model()  
print("Calibration results file:", results_file)